In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transforms and load MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)
        self.fc1 = nn.Linear(32 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(-1, 32 * 5 * 5)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Initialize the model
model = CNN().to(device)

# Training parameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1} - Loss: {running_loss / len(trainloader)}")

# Evaluate the model on the test set
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


100%|██████████| 9912422/9912422 [00:00<00:00, 86639168.87it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 119699302.20it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 22866608.46it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 11448635.08it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch 1 - Loss: 0.19766085117614504
Epoch 2 - Loss: 0.05508398247593795
Epoch 3 - Loss: 0.0381626301664866
Epoch 4 - Loss: 0.02991875745473367
Epoch 5 - Loss: 0.022885456832526286
Epoch 6 - Loss: 0.01843483566271253
Epoch 7 - Loss: 0.013932843984162622
Epoch 8 - Loss: 0.013206776999974721
Epoch 9 - Loss: 0.008907811675439402
Epoch 10 - Loss: 0.010135327792348103
Test Accuracy: 98.93%


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow_datasets as tfds

# Load MNIST dataset
(ds_train, ds_test), ds_info = tfds.load('mnist', split=['train', 'test'], shuffle_files=True, as_supervised=True, with_info=True)

# Define preprocessing functions
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255.0, label

# Apply preprocessing to train and test datasets
ds_train = ds_train.map(normalize_img)
ds_train = ds_train.batch(64).prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(normalize_img)
ds_test = ds_test.batch(64).prefetch(tf.data.AUTOTUNE)

# Define the CNN model
model = models.Sequential([
    layers.Conv2D(16, 3, activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10)
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model
model.fit(ds_train, epochs=10)

# Evaluate the model on the test set
_, test_accuracy = model.evaluate(ds_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.
Epoch 1/10
938/938 [==============================] - 23s 23ms/step - loss: 0.1972 - accuracy: 0.9418
Epoch 2/10
938/938 [==============================] - 20s 21ms/step - loss: 0.0630 - accuracy: 0.9809
Epoch 3/10
938/938 [==============================] - 19s 20ms/step - loss: 0.0448 - accuracy: 0.9863
Epoch 4/10
938/938 [==============================] - 20s 21ms/step - loss: 0.0342 - accuracy: 0.9895
Epoch 5/10
938/938 [==============================] - 20s 21ms/step - loss: 0.0265 - accuracy: 0.9921
Epoch 6/10
938/938 [==============================] - 19s 20ms/step - loss: 0.0204 - accuracy: 0.9939
Epoch 7/10
938/938 [==============================] - 20s 21ms/step - loss: 0.0154 - accuracy: 0.9952
Epoch 8/10
938/938 [==============================] - 20s 21ms/step - loss: 0.0130 - accuracy: 0.9962
Epoch 9/10
938/938 [==============================] - 19s 20ms/st